In [1]:
import pandas as pd
from tqdm import tqdm
import obspy
import numpy as np
from obspy.core.utcdatetime import UTCDateTime
import sys
import h5py 
sys.path.append("/data/wsd01/pnwstore/")

import matplotlib.pyplot as plt
from obspy.core.utcdatetime import UTCDateTime
from pnwstore.catalog import QuakeClient
from pnwstore.station import StationClient

from collections import Counter

In [2]:
quakeclient = QuakeClient("cascadia", "pnwstore")
stationclient = StationClient("cascadia", "pnwstore")
pd.set_option('display.max_columns', None)

In [36]:
df = pd.read_csv("/data/whd01/yiyu_data/PNWML/extract/noise_metadata.csv")
df

/tmp/ipykernel_486660/3722900889.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/data/whd01/yiyu_data/PNWML/extract/noise_metadata.csv")


,source_id,source_origin_time,source_latitude_deg,source_longitude_deg,source_type,source_depth_km,source_magnitude,source_magnitude_type,source_magnitude_uncertainty,source_depth_uncertainty_km,source_horizontal_uncertainty_km,station_network_code,trace_channel,station_code,station_location_code,station_latitude_deg,station_longitude_deg,station_elevation_m,trace_name,trace_sampling_rate_hz,trace_start_time,select
0,uw10565603,2002-09-21T00:55:20.960000Z,48.485,-123.126,earthquake,23.118,4.10,md,0.040,1.00,0.672,UW,BH,TTW,NaN,TBA,TBA,TBA,"bucket1$0,:3,:15001",100,2002-09-21T00:50:20.960000Z,yes
1,uw10565603,2002-09-21T00:55:20.960000Z,48.485,-123.126,earthquake,23.118,4.10,md,0.040,1.00,0.672,UW,HH,ERW,NaN,TBA,TBA,TBA,"bucket2$0,:3,:15001",100,2002-09-21T00:50:20.960000Z,yes
2,uw10565603,2002-09-21T00:55:20.960000Z,48.485,-123.126,earthquake,23.118,4.10,md,0.040,1.00,0.672,UW,BH,GNW,NaN,TBA,TBA,TBA,"bucket3$0,:3,:15001",100,2002-09-21T00:50:20.960000Z,yes
3,uw10565603,2002-09-21T00:55:20.960000Z,48.485,-123.126,earthquake,23.118,4.10,md,0.040,1.00,0.672,UW,BH,OPC,NaN,TBA,TBA,TBA,"bucket4$0,:3,:15001",100,2002-09-21T00:50:20.960000Z,yes
4,uw10563018,2002-09-13T14:59:54.050000Z,48.602,-121.973,earthquake,-0.939,2.30,md,0.040,0.08,0.050,UW,BH,GNW,NaN,TBA,TBA,TBA,"bucket5$0,:3,:15001",100,2002-09-13T14:54:54.050000Z,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51457,uw61900461,2022-12-27T10:48:24.810000Z,47.551,-122.512,earthquake,2.870,1.37,ml,0.147,0.60,0.620,UW,HH,KTSAP,NaN,TBA,TBA,TBA,"bucket1$5122,:3,:15001",100,2022-12-27T10:43:24.810000Z,yes
51458,uw61900461,2022-12-27T10:48:24.810000Z,47.551,-122.512,earthquake,2.870,1.37,ml,0.147,0.60,0.620,UW,HH,TKEY,NaN,TBA,TBA,TBA,"bucket2$5119,:3,:15001",100,2022-12-27T10:43:24.810000Z,yes
51459,uw61900461,2022-12-27T10:48:24.810000Z,47.551,-122.512,earthquake,2.870,1.37,ml,0.147,0.60,0.620,UW,HH,SKOKO,NaN,TBA,TBA,TBA,"bucket3$5116,:3,:15001",100,2022-12-27T10:43:24.810000Z,yes
51460,uw61900461,2022-12-27T10:48:24.810000Z,47.551,-122.512,earthquake,2.870,1.37,ml,0.147,0.60,0.620,UW,HH,WINDI,NaN,TBA,TBA,TBA,"bucket4$5113,:3,:15001",100,2022-12-27T10:43:24.810000Z,yes


## drop unnecessary columns

In [37]:
c = ["source_id", "source_origin_time", "source_latitude_deg", "source_longitude_deg", "source_depth_km",
     "source_magnitude", "source_magnitude_type", "source_magnitude_uncertainty", "source_depth_uncertainty_km", 
     "source_horizontal_uncertainty_km", "select"]
df.drop(columns=c, inplace = True)
df['source_type'] = 'noise'

## check location 

In [45]:
df['station_location_code'].unique()

array(['--', '00', '01'], dtype=object)

In [44]:
df.iloc[df[pd.isna(df['station_location_code'])].index, 4] = '--'
df.iloc[df[df['station_location_code'] == 0.0].index, 4] = "00"
df.iloc[df[df['station_location_code'] == '0.0'].index, 4] = "00"
df.iloc[df[df['station_location_code'] == 1.0].index, 4] = "01"
df.iloc[df[df['station_location_code'] == '1.0'].index, 4] = "01"
df.iloc[df[df['station_location_code'] == '10'].index, 4] = "01"

In [47]:
df.to_csv("/data/whd01/yiyu_data/PNWML/extract/noise_metadata.csv", sep = ',', index=False)

In [63]:
df = pd.read_csv("/data/whd01/yiyu_data/PNWML/extract/noise_metadata.csv")
df.iloc[df[(df['station_code'] == 'EPH2') & (df['trace_channel'] == 'EH')].index, 2] = "HH"
df.to_csv("/data/whd01/yiyu_data/PNWML/extract/noise_metadata.csv", sep = ',', index=False)

## add lat lon elevation

In [3]:
df = pd.read_csv("/data/whd01/yiyu_data/PNWML/extract/noise_metadata.csv")

In [4]:
error = []
for idx, i in tqdm(df.iterrows(), total = len(df)):
    if df.iloc[idx, 5] == "TBA":
        network = df.iloc[idx, 1]
        station = df.iloc[idx, 3]
        channel = df.iloc[idx, 2] + "Z"
        time = UTCDateTime(df.iloc[idx, 10])
        result = stationclient.query(network = network, station = station, channel = channel, time = time)
        
        try:
            assert len(result) >= 1
            df.iloc[idx, 5] = result.iloc[0, 5]         # latitude
            df.iloc[idx, 6] = result.iloc[0, 6]         # longitude
            df.iloc[idx, 7] = result.iloc[0, 7]         # elevation
            
        except:
            print(idx)
            error.append([idx, i])

100%|██████████| 51462/51462 [00:04<00:00, 12794.48it/s]


In [9]:
df.to_csv("/data/whd01/yiyu_data/PNWML/extract/noise_metadata.csv", sep = ',', index=False)